In [37]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [38]:
def dfdx(x,f):
    
    y= f[0]
    z=f[1]
    
    dydx = np.zeros_like(f)
    dydx[0] = z
    dydx[1] = -1*y
    
    return dydx

In [39]:
def cash_karp_core_mv(x_i,y_i,nv,h,f):
    ni = 7 
    nj = 6
    ci = np.zeros(ni)
    aij = np.zeros( (ni,nj) )
    bi = np.zeros(ni)
    bis = np.zeros(ni)
    
    ci[2] = 1./5.
    ci[3] = 3./10.
    ci[4] = 3./5.
    ci[5] = 1.
    ci[6] = 7./8.
    
    #j=1
    aij[2,1] = 1./5
    aij[3,1] = 3./40.
    aij[4,1] = 3./10.
    aij[5,1] = -11./54.
    aij[6,1] = 1631./55296.
    
    #j=2
    aij[3,2] = 9./40.
    aij[4,2] = -9./10.
    aij[5,2] = 5./2.
    aij[6,2] = 175./512.
    
    #j=3
    aij[4,3] = 6./5.
    aij[5,3] = -70./27.
    aij[6,3] = 575./13824.
    
    #j=4
    aij[5,4] = 35./27.
    aij[6,4] = 44275./110592.
    
    #j=5
    aij[6,5] = 253./4096.
    
    #bi
    bi[1] = 37./378.
    bi[2] = 0.
    bi[3] = 250./621.
    bi[4] = 125./594.
    bi[5] = 0.0
    bi[6] = 512./1771.
    
    #bis
    bis[1] = 2825./27648.
    bis[2] = 0.0
    bis[3] = 18575./48384.
    bis[4] = 13525./55296
    bis[5] = 277./14336.
    bis[6] = 1./4.
    
    #define the k array
    ki = np.zeros((ni,nv))
    
    #compute ki
    for i in range(1,ni):
        #compute xn+1 for i
        xn = x_i + ci[i]*h
        
        #compute temp y
        yn = y_i.copy()
        for j in range(1,i):
            yn += aij[i,j]*ki[j,:]
        
        #get k
        ki[i,:] = h*f(xn,yn)
        
        
        ynpo = y_i.copy()
        ynpos = y_i.copy()
        #print
        for i in range(1,ni):
            ynpo += bi[i] *ki[i,:]
            ynpos += bis[i]*ki[i,:]
        
        
        Delta = np.fabs(ynpo-ynpos)
        
        return ynpo,Delta
    
    

In [40]:
def cash_karp_mv_ad(dfdx,x_i,y_i,nv,h,tol):
    
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    imax = 10000
    
    i =0
    
    Delta = np.full(nv,2*tol)
    
    h_step = h
    
    while(Delta.max()/tol>1.0):
        y_ipo, Delta = cash_karp_core_mv(x_i,y_i,nv,h_step,dfdx)
        
        if(Delta.max()/tol>1.0):
            
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
            
        if(i>imax):
            print("Too many iterations in cash_karp_mv_ad()")
            raise StopIteration("Ending after i = ", i)
            
        i += 1
        
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9),h_step * H_NEW_FAC)
    
    return y_ipo, h_new, h_step

In [41]:
def cash_karp_mv(dfdx, a,b,y_a,tol,verbose=False):
    xi =a 
    yi = y_a.copy()
    flag=True
    h = 1.0e-4 * (b-a)
    
    imax = 1000
    
    i = 0
    
    nv = len(y_a)
    
    x=np.full(1,a)
    y=np.full( (1,nv), y_a)
    
    #loop
    while(flag):
        y_ipo, h_new, h_step = cash_karp_mv_ad(dfdx,xi,yi,nv,h,tol)
        
        h = h_new
        
        if(xi+h_step>b):
            
            h = b - xi
            
            y_ipo, h_new, h_step = cash_karp_mv_ad(dfdx, xi, yi, nv, h, tol)
            
            flag = False 
            
        xi += h_step
        yi = y_ipo.copy()
        
        x = np.append(x,xi)
        y_ipo = np.zeros((len(x),nv))
        y_ipo[0:len(x)-1,:]=y[:]
        y_ipo[-1,:] =yi[:]
        del y
        y=y_ipo
        
        if(i>=imax):
            print("Maximum iterations reached.")
            raise StopIteration("Iteration number = ", i)
        i += 1
        
        if(verbose):
            s="i = %3d\tx = %9.8f\ty = %9.8f\th = %9.8f\tb = %9.8f" % (i,xi,yi[0],h_step,b)
            print(s)
        if(xi == b):
            flag=False
            
    return x,y

In [42]:
a=0.0
b=2.0*np.pi
y_0 = np.zeros(2)
y_0[0]=0.0
y_0[1]=1.0
nv= 2

tolerance = 1.0e-6

x,y=cash_karp_mv(dfdx,a,b,y_0,tolerance,verbose=True)
plt.plot(x,y[:,0],'o',label='y(x)')
plt.plot(x,y[:,1],'o',label='z(x)')
plt.plot(x,np.sin(x),label='sin(x)')
plt.plot(x,np.cos(x),label='cos(x)')
plt.xlim([0,2*np.pi])
plt.ylim([-1.2,1.2])
plt.legend(frameon=False)

i =   1	x = 0.00021372	y = 0.00002092	h = 0.00021372	b = 6.28318531
i =   2	x = 0.00044462	y = 0.00004352	h = 0.00023090	b = 6.28318531
i =   3	x = 0.00067732	y = 0.00006630	h = 0.00023270	b = 6.28318531
i =   4	x = 0.00091019	y = 0.00008909	h = 0.00023288	b = 6.28318531
i =   5	x = 0.00114309	y = 0.00011189	h = 0.00023289	b = 6.28318531
i =   6	x = 0.00137598	y = 0.00013469	h = 0.00023290	b = 6.28318531
i =   7	x = 0.00160888	y = 0.00015748	h = 0.00023290	b = 6.28318531
i =   8	x = 0.00184177	y = 0.00018028	h = 0.00023290	b = 6.28318531
i =   9	x = 0.00207467	y = 0.00020308	h = 0.00023290	b = 6.28318531
i =  10	x = 0.00230756	y = 0.00022587	h = 0.00023290	b = 6.28318531
i =  11	x = 0.00254046	y = 0.00024867	h = 0.00023290	b = 6.28318531
i =  12	x = 0.00277335	y = 0.00027147	h = 0.00023290	b = 6.28318531
i =  13	x = 0.00300625	y = 0.00029426	h = 0.00023290	b = 6.28318531
i =  14	x = 0.00323915	y = 0.00031706	h = 0.00023290	b = 6.28318531
i =  15	x = 0.00347204	y = 0.00033986	h = 0.0002

i = 332	x = 0.07730058	y = 0.00756639	h = 0.00023290	b = 6.28318531
i = 333	x = 0.07753348	y = 0.00758918	h = 0.00023290	b = 6.28318531
i = 334	x = 0.07776638	y = 0.00761198	h = 0.00023290	b = 6.28318531
i = 335	x = 0.07799928	y = 0.00763478	h = 0.00023290	b = 6.28318531
i = 336	x = 0.07823218	y = 0.00765757	h = 0.00023290	b = 6.28318531
i = 337	x = 0.07846509	y = 0.00768037	h = 0.00023290	b = 6.28318531
i = 338	x = 0.07869799	y = 0.00770317	h = 0.00023290	b = 6.28318531
i = 339	x = 0.07893089	y = 0.00772596	h = 0.00023290	b = 6.28318531
i = 340	x = 0.07916379	y = 0.00774876	h = 0.00023290	b = 6.28318531
i = 341	x = 0.07939670	y = 0.00777156	h = 0.00023290	b = 6.28318531
i = 342	x = 0.07962960	y = 0.00779435	h = 0.00023290	b = 6.28318531
i = 343	x = 0.07986250	y = 0.00781715	h = 0.00023290	b = 6.28318531
i = 344	x = 0.08009540	y = 0.00783995	h = 0.00023290	b = 6.28318531
i = 345	x = 0.08032830	y = 0.00786274	h = 0.00023290	b = 6.28318531
i = 346	x = 0.08056121	y = 0.00788554	h = 0.0002

i = 630	x = 0.14670765	y = 0.01435978	h = 0.00023292	b = 6.28318531
i = 631	x = 0.14694057	y = 0.01438258	h = 0.00023292	b = 6.28318531
i = 632	x = 0.14717348	y = 0.01440537	h = 0.00023292	b = 6.28318531
i = 633	x = 0.14740640	y = 0.01442817	h = 0.00023292	b = 6.28318531
i = 634	x = 0.14763932	y = 0.01445097	h = 0.00023292	b = 6.28318531
i = 635	x = 0.14787224	y = 0.01447376	h = 0.00023292	b = 6.28318531
i = 636	x = 0.14810516	y = 0.01449656	h = 0.00023292	b = 6.28318531
i = 637	x = 0.14833808	y = 0.01451936	h = 0.00023292	b = 6.28318531
i = 638	x = 0.14857100	y = 0.01454215	h = 0.00023292	b = 6.28318531
i = 639	x = 0.14880392	y = 0.01456495	h = 0.00023292	b = 6.28318531
i = 640	x = 0.14903684	y = 0.01458775	h = 0.00023292	b = 6.28318531
i = 641	x = 0.14926976	y = 0.01461054	h = 0.00023292	b = 6.28318531
i = 642	x = 0.14950268	y = 0.01463334	h = 0.00023292	b = 6.28318531
i = 643	x = 0.14973560	y = 0.01465614	h = 0.00023292	b = 6.28318531
i = 644	x = 0.14996852	y = 0.01467893	h = 0.0002

i = 938	x = 0.21845094	y = 0.02138114	h = 0.00023295	b = 6.28318531
i = 939	x = 0.21868389	y = 0.02140394	h = 0.00023295	b = 6.28318531
i = 940	x = 0.21891684	y = 0.02142673	h = 0.00023295	b = 6.28318531
i = 941	x = 0.21914979	y = 0.02144953	h = 0.00023295	b = 6.28318531
i = 942	x = 0.21938274	y = 0.02147233	h = 0.00023295	b = 6.28318531
i = 943	x = 0.21961569	y = 0.02149512	h = 0.00023295	b = 6.28318531
i = 944	x = 0.21984864	y = 0.02151792	h = 0.00023295	b = 6.28318531
i = 945	x = 0.22008158	y = 0.02154072	h = 0.00023295	b = 6.28318531
i = 946	x = 0.22031453	y = 0.02156351	h = 0.00023295	b = 6.28318531
i = 947	x = 0.22054748	y = 0.02158631	h = 0.00023295	b = 6.28318531
i = 948	x = 0.22078043	y = 0.02160911	h = 0.00023295	b = 6.28318531
i = 949	x = 0.22101338	y = 0.02163190	h = 0.00023295	b = 6.28318531
i = 950	x = 0.22124633	y = 0.02165470	h = 0.00023295	b = 6.28318531
i = 951	x = 0.22147928	y = 0.02167750	h = 0.00023295	b = 6.28318531
i = 952	x = 0.22171223	y = 0.02170029	h = 0.0002

StopIteration: ('Iteration number = ', 1000)